# Ion Exchange Simulation - H-WAC → Degasser → Na-WAC

This notebook performs detailed simulation of the H-WAC flowsheet for high temporary hardness removal.

## Process Description
1. **H-WAC**: Removes temporary hardness (Ca/Mg bicarbonates) using acid regeneration
2. **Degasser**: Strips CO2 released from bicarbonate conversion
3. **Na-WAC**: Polishing step for residual hardness

Best for: Waters with >90% temporary hardness

In [ ]:
# Parameters cell - papermill will inject values here
import json

# Configuration parameters
configuration = None  # Will be injected by papermill
water_analysis = None  # Will be injected by papermill
simulation_options = {
    "model_type": "direct",
    "time_steps": 100,
    "breakthrough_criteria": {"hardness_mg_L_CaCO3": 5.0}
}

In [ ]:
# Import required modules
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Add project root to path
if configuration and 'project_root' in configuration:
    sys.path.insert(0, configuration['project_root'])

from tools.ix_simulation_direct import simulate_ix_system_direct
from tools.schemas import IXSimulationInput, IXConfigurationOutput, MCASWaterComposition

## Water Analysis

In [ ]:
# Display water analysis
if water_analysis:
    print("Feed Water Composition:")
    print(f"Flow: {water_analysis['flow_m3_hr']} m³/hr")
    print(f"pH: {water_analysis['pH']}")
    print(f"Temperature: {water_analysis['temperature_celsius']}°C")
    
    # Calculate hardness
    ca = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0)
    mg = water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)
    hco3 = water_analysis['ion_concentrations_mg_L'].get('HCO3_-', 0)
    
    total_hardness = ca * 2.5 + mg * 4.1
    alkalinity = hco3 / 1.22
    temp_hardness = min(total_hardness, alkalinity)
    perm_hardness = max(0, total_hardness - alkalinity)
    
    print(f"\nWater Chemistry:")
    print(f"Total Hardness: {total_hardness:.1f} mg/L as CaCO3")
    print(f"Alkalinity: {alkalinity:.1f} mg/L as CaCO3")
    print(f"Temporary Hardness: {temp_hardness:.1f} mg/L ({temp_hardness/total_hardness*100:.0f}%)")
    print(f"Permanent Hardness: {perm_hardness:.1f} mg/L ({perm_hardness/total_hardness*100:.0f}%)")

## Run Simulation

In [ ]:
# Prepare simulation input
if configuration and water_analysis:
    # Create proper schema objects
    config_obj = IXConfigurationOutput(**configuration)
    water_obj = MCASWaterComposition(**water_analysis)
    
    sim_input = IXSimulationInput(
        configuration=config_obj,
        water_analysis=water_obj,
        breakthrough_criteria=simulation_options['breakthrough_criteria'],
        simulation_options=simulation_options
    )
    
    print("Running H-WAC flowsheet simulation...")
    result = simulate_ix_system_direct(sim_input)
    print(f"Simulation status: {result.status}")

## Performance Results

In [ ]:
# Display performance metrics for each vessel
if result and result.ix_performance:
    for vessel_name, metrics in result.ix_performance.items():
        print(f"\n{vessel_name} Performance:")
        print(f"  Breakthrough: {metrics.bed_volumes_treated:.0f} BV ({metrics.breakthrough_time_hours:.1f} hours)")
        print(f"  Regenerant: {metrics.regenerant_consumption_kg:.1f} kg {metrics.regenerant_chemical}")
        print(f"  Capacity utilization: {metrics.capacity_utilization_percent:.0f}%")
        print(f"  Average leakage: {metrics.average_hardness_leakage_mg_L:.1f} mg/L")

## Water Quality Progression

In [ ]:
# Plot water quality through treatment stages
if result and result.water_quality_progression:
    stages = [p.stage for p in result.water_quality_progression]
    hardness = [p.hardness_mg_L_CaCO3 for p in result.water_quality_progression]
    ph = [p.pH for p in result.water_quality_progression]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Hardness reduction
    ax1.plot(range(len(stages)), hardness, 'b-o')
    ax1.set_xticks(range(len(stages)))
    ax1.set_xticklabels(stages, rotation=45, ha='right')
    ax1.set_ylabel('Hardness (mg/L as CaCO3)')
    ax1.set_title('Hardness Reduction Through Treatment')
    ax1.grid(True)
    
    # pH changes
    ax2.plot(range(len(stages)), ph, 'r-o')
    ax2.set_xticks(range(len(stages)))
    ax2.set_xticklabels(stages, rotation=45, ha='right')
    ax2.set_ylabel('pH')
    ax2.set_title('pH Changes Through Treatment')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

## Economics Summary

In [ ]:
# Display economic results
if result and result.economics:
    econ = result.economics
    print("Economic Analysis:")
    print(f"  Capital cost: ${econ.get('capital_cost', 0):,.0f}")
    print(f"  Annual OPEX: ${econ.get('operating_cost_annual', 0):,.0f}")
    print(f"  Cost per m³: ${econ.get('cost_per_m3', 0):.3f}")
    
    # Chemical consumption
    if result.chemical_consumption:
        print("\nChemical Consumption (per day):")
        for chem, amount in result.chemical_consumption.items():
            print(f"  {chem}: {amount:.1f}")

## Degasser Performance

In [ ]:
# Display degasser results
if result and result.degasser_performance:
    deg = result.degasser_performance
    print("Degasser Performance:")
    print(f"  Influent CO2: {deg.get('influent_CO2_mg_L', 0):.1f} mg/L")
    print(f"  Effluent CO2: {deg.get('effluent_CO2_mg_L', 0):.1f} mg/L")
    print(f"  Removal: {deg.get('removal_percent', 0):.0f}%")
    print(f"  pH increase: {deg.get('effluent_pH', 0) - result.water_quality_progression[1].pH:.1f}")

## Recommendations

In [ ]:
# Generate operational recommendations
if result:
    print("Operational Recommendations:")
    
    # H-WAC specific recommendations
    print("\n1. H-WAC Operation:")
    print("   - Monitor effluent pH carefully (should drop to 4.5-5.5)")
    print("   - Use HCl for regeneration (5% solution typical)")
    print("   - Ensure proper degassing after H-WAC to prevent CO2 carryover")
    
    print("\n2. Degasser Operation:")
    print("   - Maintain air:water ratio at 45:1")
    print("   - Monitor packing for fouling/scaling")
    print("   - Target <5 mg/L CO2 in effluent")
    
    print("\n3. Na-WAC Polisher:")
    print("   - Expect longer run times (polishing only)")
    print("   - Can use NaCl for regeneration")
    print("   - Monitor for Na+ leakage")
    
    if result.recommendations:
        print("\n4. Additional Recommendations:")
        for rec in result.recommendations:
            print(f"   - {rec}")

In [ ]:
# Prepare results for extraction
results = result.dict() if result else {"status": "error", "message": "No simulation results"}
results